In [264]:
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None

Read data

In [265]:
df_races = pd.read_csv('01_raw/races.csv')
df_circuits = pd.read_csv('01_raw/circuits.csv')
df_drivers = pd.read_csv('01_raw/drivers.csv')
df_results = pd.read_csv('01_raw/results.csv')
# df_cons_res = pd.read_csv('01_raw/constructor_results.csv')

In [266]:
df_results.head(1)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.000,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1


In [267]:
df_races.head(1)

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,29/3/2009,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...


In [268]:
df_circuits.head(1)

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.850,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...


In [269]:
df_drivers.head(1)

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton


Create master dataset

In [270]:
df_drivers = df_drivers[['driverId', 'driverRef', 'nationality']]
df_drivers.columns = ['driverId', 'driverRef', 'driver_nationality']
df_results['driverId'] = df_results['driverId'].astype(str)
df_drivers['driverId'] = df_drivers['driverId'].astype(str)
df = df_results.merge(df_drivers, on='driverId', how='left', validate='m:1')

In [271]:
df_races = df_races[['raceId', 'year', 'round', 'name', 'date', 'time', 'circuitId']]
df_races.columns = ['raceId', 'race_year', 'round', 'race_name', 'race_date', 'race_time', 'circuitId']
df['raceId'] = df['raceId'].astype(str)
df_races['raceId'] = df_races['raceId'].astype(str)
df = df.merge(df_races, on='raceId', how='left')

some kind of preprocessing necessary for lapping
perhaps marked as lapped if dif in laps is +1
lapped x2 if dif in laps is +2 
etc
if more than 3 laps, classify as DNF

In [272]:
df_circuits = df_circuits[['circuitId', 'name', 'country', 'alt']]
df_circuits.columns = ['circuitId', 'circuit_name', 'circuit_country', 'circuit_alt']
df = df.merge(df_circuits, on='circuitId', how='left', validate='m:1')

In [273]:

df['ms_num'] = df['milliseconds']
# df['ms_num'].replace({'\\N': '99999999'}, inplace=True)
df['ms_num'] = pd.to_numeric(df['ms_num'], errors='coerce')
# df = df.sort_values(['raceId', ''])
df['ms_num'] = df['ms_num'].fillna(method='ffill')
df['avg_lap_ms'] = df['ms_num']/df['laps']

# add column with fastest lap per race
df_fastest_time = df.groupby('raceId')['ms_num'].min().reset_index()
df_fastest_time.columns = ['raceId', 'fastest_time']

df = df.merge(df_fastest_time, on='raceId', how='left', validate='m:1')

# add column with average lap of the fastest racer
df_avg_top = df.groupby('raceId')['avg_lap_ms'].min().reset_index()
df_avg_top.columns = ['raceId', 'top_one_avg_lap']
df = df.merge(df_avg_top, on='raceId', how='left', validate='m:1')

# add column with amount of laps per race
df_tot_laps = df.groupby('raceId')['laps'].max().reset_index()
df_tot_laps.columns = ['raceId', 'total_laps']
df = df.merge(df_tot_laps, on='raceId', how='left', validate='m:1')

df['lap_delta'] = df['total_laps'] - df['laps']

thresh = 3
df.loc[(df['lap_delta'] <= thresh) & (df['lap_delta'] > 0), 'ms_num'] = df['ms_num'] + df['top_one_avg_lap'] * df['lap_delta']
df.loc[df['lap_delta'] > thresh, 'ms_num'] = 999999999


In [274]:
df[df['raceId'] == '990'][['position', 'milliseconds', 'ms_num', 'laps', 'avg_lap_ms', 'fastest_time', 'top_one_avg_lap', 'total_laps', 'lap_delta']]

,position,milliseconds,ms_num,laps,avg_lap_ms,fastest_time,top_one_avg_lap,total_laps,lap_delta
23797,1,5521940,5521940.000,57,96876.140,5521940.000,96876.140,57,0
23798,2,5522639,5522639.000,57,96888.404,5521940.000,96876.140,57,0
23799,3,5528452,5528452.000,57,96990.386,5521940.000,96876.140,57,0
23800,4,5584174,5584174.000,57,97967.965,5521940.000,96876.140,57,0
23801,5,5596986,5596986.000,57,98192.737,5521940.000,96876.140,57,0
23802,6,5620964,5620964.000,57,98613.404,5521940.000,96876.140,57,0
23803,7,\N,5717840.140,56,100374.357,5521940.000,96876.140,57,1
23804,8,\N,5717840.140,56,100374.357,5521940.000,96876.140,57,1
23805,9,\N,5717840.140,56,100374.357,5521940.000,96876.140,57,1
23806,10,\N,5717840.140,56,100374.357,5521940.000,96876.140,57,1


Output master dataset

In [275]:
df.to_csv('02_intermediate/pipeline_output.csv', index=False)